#### Slowly Changing Dimensions (SCD) Type 2 Implementation

In [0]:
%sql

DROP TABLE IF EXISTS emp;

CREATE TABLE emp(
  emp_id int,
  emp_name string,
  emp_loc string,
  active boolean,
  effective_date timestamp,
  end_date timestamp
);

SELECT * FROM emp;

emp_id,emp_name,emp_loc,active,effective_date,end_date


In [0]:
%sql

INSERT INTO emp VALUES
(1,'PrasadP','Bangalore',1,'2023-08-08','9999-12-31'),
(2,'Satish','Bangalore',1,'2023-08-08','9999-12-31'),
(3,'PrasadA','Bangalore',1,'2023-08-08','9999-12-31');

SELECT * FROM emp;

emp_id,emp_name,emp_loc,active,effective_date,end_date
1,PrasadP,Bangalore,true,2023-08-08T00:00:00Z,9999-12-31T00:00:00Z
2,Satish,Bangalore,true,2023-08-08T00:00:00Z,9999-12-31T00:00:00Z
3,PrasadA,Bangalore,true,2023-08-08T00:00:00Z,9999-12-31T00:00:00Z


In [0]:
%sql

DROP TABLE IF EXISTS emp_source1;

CREATE TABLE emp_source1(
  emp_id int,
  emp_name string,
  emp_loc string,
  effective_date timestamp
);

SELECT * FROM emp_source1;

emp_id,emp_name,emp_loc,effective_date


In [0]:
%sql

INSERT INTO emp_source1 VALUES
(4,'Ankit','Bangalore','2023-08-08'),
(1,'PrasadP','Shridi','2023-11-11'),
(3,'PrasadA','Latur','2023-11-11'),
(5,'Sriram','Bangalore','2023-08-08');

SELECT * FROM emp_source1;

emp_id,emp_name,emp_loc,effective_date
4,Ankit,Bangalore,2023-08-08T00:00:00Z
1,PrasadP,Shridi,2023-11-11T00:00:00Z
3,PrasadA,Latur,2023-11-11T00:00:00Z
5,Sriram,Bangalore,2023-08-08T00:00:00Z


In [0]:
%sql

-- Implementation of staging table to perform MERGE operation later on to target table

SELECT emp_id as merge_key, emp_source1.*
FROM emp_source1
UNION ALL
SELECT NULL as merge_key, emp_source1.*
FROM emp_source1
JOIN emp
ON emp_source1.emp_id = emp.emp_id
WHERE emp.active = 1 and emp_source1.emp_loc <> emp.emp_loc;

merge_key,emp_id,emp_name,emp_loc,effective_date
4,4,Ankit,Bangalore,2023-08-08T00:00:00Z
1,1,PrasadP,Shridi,2023-11-11T00:00:00Z
3,3,PrasadA,Latur,2023-11-11T00:00:00Z
5,5,Sriram,Bangalore,2023-08-08T00:00:00Z
null,1,PrasadP,Shridi,2023-11-11T00:00:00Z
null,3,PrasadA,Latur,2023-11-11T00:00:00Z


In [0]:
%sql

-- now using the above table to update and insert to apply SCD type2

-- first MERGE operation

MERGE INTO emp as tgt
USING (
  SELECT emp_id as merge_key, emp_source1.*
FROM emp_source1
UNION ALL
SELECT NULL as merge_key, emp_source1.*
FROM emp_source1
JOIN emp
ON emp_source1.emp_id = emp.emp_id
WHERE emp.active = 1 and emp_source1.emp_loc <> emp.emp_loc
) as src
ON tgt.emp_id = src.merge_key
WHEN MATCHED AND tgt.active = 1 AND tgt.emp_loc <> src.emp_loc THEN
UPDATE SET tgt.active = 0, tgt.end_date=src.effective_date
WHEN NOT MATCHED THEN
INSERT (emp_id,emp_name,emp_loc,active,effective_date,end_date)
VALUES (emp_id,emp_name,emp_loc,1,effective_date,'9999-12-31');

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
6,2,0,4


In [0]:
%sql

SELECT * FROM emp;

emp_id,emp_name,emp_loc,active,effective_date,end_date
4,Ankit,Bangalore,true,2023-08-08T00:00:00Z,9999-12-31T00:00:00Z
1,PrasadP,Bangalore,false,2023-08-08T00:00:00Z,2023-11-11T00:00:00Z
3,PrasadA,Bangalore,false,2023-08-08T00:00:00Z,2023-11-11T00:00:00Z
5,Sriram,Bangalore,true,2023-08-08T00:00:00Z,9999-12-31T00:00:00Z
2,Satish,Bangalore,true,2023-08-08T00:00:00Z,9999-12-31T00:00:00Z
1,PrasadP,Shridi,true,2023-11-11T00:00:00Z,9999-12-31T00:00:00Z
3,PrasadA,Latur,true,2023-11-11T00:00:00Z,9999-12-31T00:00:00Z


In [0]:
%sql

-- creating second source table

DROP TABLE IF EXISTS emp_source2;

CREATE TABLE emp_source2(
  emp_id INT,
  emp_name string,
  emp_loc string,
  effective_date timestamp
);

SELECT * FROM emp_source2;

emp_id,emp_name,emp_loc,effective_date


In [0]:
%sql

-- inserting values into our source table

INSERT INTO emp_source2 VALUES
(4,'Ankit','Jamshedpur','2023-11-18'),
(1,'PrasadP','Hyderabad','2023-11-23');

SELECT * FROM emp_source2;

emp_id,emp_name,emp_loc,effective_date
4,Ankit,Jamshedpur,2023-11-18T00:00:00Z
1,PrasadP,Hyderabad,2023-11-23T00:00:00Z


In [0]:
%sql

-- MERGING the second source table into our target emp table

MERGE INTO emp as tgt
USING
(
  SELECT emp_id as merge_key, emp_source2.*
  FROM emp_source2
  UNION ALL
  SELECT NULL as merge_key, emp_source2.*
  FROM emp_source2
  JOIN emp
  ON emp_source2.emp_id=emp.emp_id 
  WHERE emp.active=1 AND emp_source2.emp_loc <> emp.emp_loc
) as src
ON tgt.emp_id = src.merge_key
WHEN MATCHED AND tgt.active=1 AND tgt.emp_loc <> src.emp_loc AND tgt.effective_date <> src.effective_date THEN
UPDATE SET tgt.active=0, tgt.end_date=src.effective_date
WHEN NOT MATCHED THEN
INSERT (emp_id,emp_name,emp_loc,active,effective_date,end_date)
VALUES (emp_id,emp_name,emp_loc,1,effective_date,'9999-12-31');


num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
4,2,0,2


In [0]:
%sql

SELECT * FROM emp;

emp_id,emp_name,emp_loc,active,effective_date,end_date
1,PrasadP,Hyderabad,true,2023-11-23T00:00:00Z,9999-12-31T00:00:00Z
2,Satish,Bangalore,true,2023-08-08T00:00:00Z,9999-12-31T00:00:00Z
4,Ankit,Jamshedpur,true,2023-11-18T00:00:00Z,9999-12-31T00:00:00Z
1,PrasadP,Bangalore,false,2023-08-08T00:00:00Z,2023-11-11T00:00:00Z
3,PrasadA,Bangalore,false,2023-08-08T00:00:00Z,2023-11-11T00:00:00Z
5,Sriram,Bangalore,true,2023-08-08T00:00:00Z,9999-12-31T00:00:00Z
3,PrasadA,Latur,true,2023-11-11T00:00:00Z,9999-12-31T00:00:00Z
4,Ankit,Bangalore,false,2023-08-08T00:00:00Z,2023-11-18T00:00:00Z
1,PrasadP,Shridi,false,2023-11-11T00:00:00Z,2023-11-23T00:00:00Z
